# Répartition du temps de parole entre les hommes et les femmes dans les médias français

## Imports des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bs4
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

import warnings
warnings.filterwarnings("ignore")

## Structure des données : quelques statistiques descriptives 

In [ ]:
# Télécharger depuis la source
# df_stats = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/db598503-d6e5-4e89-8214-167bd239d9e9")

# Version locale
df_stats = pd.read_csv("data/temps_parole_media.csv")

print(f"Format de df_stats : {df_stats.shape}")

In [ ]:
# Échantillon aléatoire du DataFrame
df_stats.sample(10)

Cette DataFrame s'intitule "temps de musique et temps de parole des hommes et des femmes à la TV et à la radio".  Réalisée pour une étude de l'Institut National de l'Audiovisuel (INA) sur la répartition du temps de parole entre les hommes et les femmes à la télévision et à la radio, elle synthétise l’analyse de 1 078 801 heures de programmes diffusés à la télévision et à la radio entre 1995 et 2019. 

Les temps de parole des hommes et des femmes ont été mesurés par une intelligence artificielle développée par l'INA, le logiciel InaSpeechSegmenter basé sur des algorithmes d'apprentissage automatique. Entraînés sur un grand nombre de musique, de voix de femmes et d'hommes afin de détecter les zones de musique et les zones de parole contenues dans les documents audiovisuels, ce logiciel a permis d'analyser un très grand nombre d'heures d'audience.

Comme étudié ci-dessous, cette base de données analyse 21 stations radio: Chérie FM, Europe 1, France Bleu, France Culture, France Info, France Inter, France Musique, Fun Radio, Mouv’, NRJ, Nostalgie, RFM, RMC, RTL, RTL 2, Radio Classique, Radio France Internationale, Rire et Chansons, Skyrock, Sud Radio et Virgin Radio.

D'autre part, elle comprend 34 chaînes de TV: Arte, Animaux, BFM TV, Canal+, Canal+ Sport, Chasse et pêche, Chérie 25, Comédie+, D8/C8, Euronews, Eurosport France, France 2, France 24, France 3, France 5, France O, Histoire, I-Télé/CNews, L'Equipe 21, LCI, LCP/Public Sénat, La chaîne Météo, M6, Monte Carlo TMC, NRJ 12, Paris Première, Planète+, TF1, TV Breizh, TV5 Monde, Toute l'Histoire, Téva, Voyage, W9. 

In [ ]:
# Création de la colonne "year" et de la colonne "months" à partir de la colonne "date"
df_stats["year"] = pd.to_datetime(df_stats["date"]).dt.year
df_stats["month"] = pd.to_datetime(df_stats["date"]).dt.month

In [ ]:
# nombre d'heures analysées
df_stats["hour"].count()

In [ ]:
print(f"Date miniumum : {df_stats['date'].min()}")
print(f"Date maximum  : {df_stats['date'].max()}")
print(
    f'Le nombre de station radio est : {df_stats.loc[df_stats["media_type"] == "radio", "channel_name"].nunique()}'
)
print(
    f'Le nombre de chaînes tv est : {df_stats.loc[df_stats["media_type"] == "tv", "channel_name"].nunique()}'
)

In [ ]:
# nombre d'entrées radio et nombre d'entrées tv
print("Types de media:")
df_stats["media_type"].value_counts()

In [ ]:
# nombre d'entrées par stations radio au sein du "media_type" radio
print("Canaux radio :")
df_stats.loc[df_stats["media_type"] == "radio", "channel_name"].value_counts()

In [ ]:
# nombre d'entrées par chaines tv au sein du "media_type" tv
print("Canaux tv :")
df_stats.loc[df_stats["media_type"] == "tv", "channel_name"].value_counts()

## Analyse descriptive des données

À ce stade, la granularité temporelle est l'heure. Chaque ligne rend compte de ce qu'il s'est passé, pendant l'heure donnée, un certain jour, sur un canal donné.
Le temps est en secondes (1h = 3600 secondes). 

### Calcul des ratios

In [ ]:
# création d'une fonction ratio qui permet d'ajouter quatres nouvelles colonnes à la base de données :
# women_expression_rate, men_expression_rate, music_rate, speech_rate

def ratio(df):
    df = df.copy()
    df["women_expression_rate"] = df["female_duration"] / (
        df["female_duration"] + df["male_duration"]
    )
    df["men_expression_rate"] = df["male_duration"] / (
        df["female_duration"] + df["male_duration"]
    )
    df["music_rate"] = df["music_duration"] / (
        df["female_duration"] + df["male_duration"] + df["music_duration"]
    )
    df["speech_rate"] = (df["female_duration"] + df["male_duration"]) / (
        df["female_duration"] + df["male_duration"] + df["music_duration"]
    )
    return df

### Agrégation à l'année et par chaîne

Changement de granularité. Le niveau d'étude n'est plus la journée désormais mais l'année.

In [ ]:
# On a précédemment ajouté une colonne year au tableau.
# On aggrège par année, ie on rassemble au sein d'une même ligne toutes les différentes lignes ayant le même "media_type", "channel_code", "channel_name", "is_public_channel" et "year"
# On somme sur ces lignes les male_duration, female_duration, music_duration pour avoir un total par jour et on compte les "hour" (car le chiffre reporté dans la colonne heure correspond à l'heure d'enregistrement de l'écoute)
# Cela permet de définir à nouveau une nouvelle base de données : df_year_level

df_year_level = (
    df_stats.groupby(
        [
            "media_type",
            "channel_code",
            "channel_name",
            "is_public_channel",
            "year",
        ]
    )
    .agg(
        {
            "male_duration": "sum",
            "female_duration": "sum",
            "music_duration": "sum",
            "hour": "count",
        }
    )
    .reset_index()
    .rename(
        columns={
            "hour": "nb_hours_analyzed",
        }
    )
)
df_year_level.head(5)

In [ ]:
# Ratio par chaîne et par année
df_year_level = ratio(df_year_level)
df_year_level.head(5)

In [ ]:
sns.relplot(
    x="year",
    y="women_expression_rate",
    hue="channel_name",
    dashes=False,
    markers=True,
    kind="line",
    data=df_year_level[df_year_level["channel_name"].str.startswith("France")],
)
plt.title("Evolution of the women expression rate along the years")
plt.show()

L'évolution des chaines de télévision du groupe France télévisions et du groupe Radio France présente, sur la période 1995-2019, des women_expression_rate tous compris entre 0.17 et 0.35, à l'exception de deux chaînes. On est dans l'ensemble très loin de la parité, à l'exception de France 24 qui se situe toutes années confondues bien au-dessus des autres chaines/stations.

### Agrégation par chaine seulement et distinction public et privé

Quelles chaines donnent le plus la parole aux femmes ?\
Quelles stations radio donnent le plus la parole aux femmes ?\
Qu'en est-il des chaines de sport?

Remarque : dans la DataFrame sont distinguées channel_name et channel_code. Une même chaîne/station peut avoir deux channel_code du fait d'un changement de nom entre 1995 et 2019 mais elles n'auront qu'un channel_name. 
Donc le regroupement est fait par channel_name et non channel_code pour n'avoir qu'une ligne par chaîne. 

In [ ]:
# But: obtenir désormais une entrée par chaîne

df_channel_level = (
    df_stats.groupby(
        [
            "media_type",
            "channel_name",
            "is_public_channel",
        ]
    )
    .agg(
        {
            "male_duration": "sum",
            "female_duration": "sum",
            "music_duration": "sum",
            "hour": "count",
        }
    )
    .reset_index()
    .rename(
        columns={
            "hour": "nb_hours_analyzed",
        }
    )
)
df_channel_level.head(5)

In [ ]:
# Ratio par chaîne
df_channel_level = ratio(df_channel_level)
df_channel_level.head(5)

In [ ]:
# Classement des chaînes par ordre décroissant du women_expression_rate
# on reset.index pour clarifier le classement
df_channel_sorted = df_channel_level.sort_values(
    by="women_expression_rate", ascending=False
).reset_index(drop=True)
df_channel_sorted.head(5)

In [ ]:
# pour rechercher le classement d'une station/chaîne dans la DataFrame des classements
# dans search on peut mettre n'importe quelle des chaînes tv ou radio présente dans la DataFrame
# examples
search = "France Inter"
df_channel_sorted.loc[df_channel_sorted["channel_name"].str.contains(search)]

# ou
search = "D8"
df_channel_sorted.loc[df_channel_sorted["channel_name"].str.contains(search)]

In [ ]:
# Classement parmi les radios uniquement
df_channel_sorted_radio_only = df_channel_sorted.loc[
    df_channel_sorted["media_type"] == "radio"
]
df_channel_sorted_radio_only

# Chérie FM
#df_channel_sorted_radio_only.head(5)

# Skyrock
df_channel_sorted_radio_only.tail(5)

In [ ]:
# Classement parmi les chaines tv uniquement
df_channel_sorted_tv_only = df_channel_sorted.loc[
    df_channel_sorted["media_type"] == "tv"
]
df_channel_sorted_tv_only

# Téva
df_channel_sorted_tv_only.head()
# Canal+Sport
df_channel_sorted_tv_only.tail()

In [ ]:
# Classement des chaînes privées en terme de women_expression_rate
df_channel_sorted.loc[df_channel_sorted["is_public_channel"] == False]

# Classement des chaînes publiques en terme de women_expression_rate
df_channel_sorted.loc[df_channel_sorted["is_public_channel"] == True]

In [ ]:
# Visualisation graphique, classement des chaînes

sns.catplot(
    x="channel_name",
    y="women_expression_rate",
    data=df_channel_sorted,
    col="media_type",
    row="is_public_channel",
    kind="bar",
    height=10,
    palette="Spectral",
).set_xticklabels(rotation=90)
plt.show()

### Étude de l'évolution temporelle du temps de parole des femmes à différents niveaux 

#### Évolution temporelle du taux d'expression des femmes global (radio et TV confondus)

Quelle est l'évolution annuelle du taux de parole des femmes entre 1995 et 2019 ? 

In [ ]:
# On groupe par année

df_years = (
    df_stats.groupby(
        [
            "year",
        ]
    )
    .agg(
        {
            "male_duration": "sum",
            "female_duration": "sum",
            "music_duration": "sum",
            "hour": "count",
        }
    )
    .reset_index()
    .rename(
        columns={
            "hour": "nb_hours_analyzed",
        }
    )
)

In [ ]:
# Ratio par année
df_years = ratio(df_years)

In [ ]:
df_years_sorted = df_years.sort_values(
    by="women_expression_rate", ascending=True
).reset_index(drop=True)
df_years_sorted.head(5)

Entre 1995 et 2020, le taux d'expression des femmes dans les médias (tout média confondu) croit dans l'ensemble sur la période. 

In [ ]:
# visualisation :

sns.relplot(
    x="year",
    y="women_expression_rate",
    data=df_years_sorted,
    kind="line",
)

plt.title("Evolution du taux de parole des femmes au cours du temps")
plt.show()

In [ ]:
# Régression :

sns.regplot(
    x="year",
    y="women_expression_rate",
    data=df_years_sorted,
)
plt.show()

### Sur quelles thématiques les femmes parlent-elles le plus ?  

#### Les radio musicales sont-elles plus féminines ou masculine en terme de temps de parole ? 

In [ ]:
df_radio = df_stats[df_stats["media_type"] == "radio"].copy()

# en regardant une station radio musicale on constate que le taux de musique est proche de 0.7
# création d'une indicatrice indiquant si la station radio est musicale ou non.
# le critère est d'avoir un taux de musique > 0.7
# on ajoute musique rate à la base de donnée df_radio

df_radio["music_rate"] = df_radio["music_duration"] / (
    df_radio["female_duration"] + df_radio["male_duration"] + df_radio["music_duration"]
)

df_radio["musical_channel"] = df_radio["music_rate"] > 0.7
df_radio.head(5)

In [ ]:
# Chérie FM par exemple est considérée comme une station radio musicale
df_radio.iloc[0, -1]

In [ ]:
df_radio_musical_channel = (
    df_radio.groupby(
        [
            "musical_channel",
        ]
    )
    .agg(
        {
            "female_duration": "sum",
            "male_duration": "sum",
            "music_duration": "sum",
            "hour": "count",
        }
    )
    .reset_index()
    .rename(
        columns={
            "hour": "nb_hours_analyzed",
        }
    )
)
df_radio_musical_channel

In [ ]:
# recréeation du women_expression_rate
df_radio_musical_channel["women_expression_rate"] = df_radio_musical_channel["female_duration"] / (df_radio_musical_channel["female_duration"] + df_radio_musical_channel["male_duration"])
df_radio_musical_channel

On constate que les femmes parlent un peu plus sur les chaînes musicales que sur les autres chaines.

In [ ]:
musical_channel = (
    ratio(
        df_radio_musical_channel.groupby(["musical_channel"])
        .agg(
            {
                "female_duration": "sum",
                "male_duration": "sum",
                "music_duration":"sum",
                "nb_hours_analyzed": "count",
            }
        )
        .reset_index()
    )
    .set_index(["musical_channel"])[["women_expression_rate", "men_expression_rate"]]
)
musical_channel

In [ ]:
musical_channel.T.plot.pie(subplots=True, legend=False, figsize=(10, 100), autopct="%.2f")
plt.legend( loc = 'best' )
plt.title("Taux d'expression masculin et féminin selon le type de chaîne (musical-True ou non-False)")

On voit graphiquement que sur les chaînes dites musicales, les femmes parlent un peu plus que sur les autres.

#### Etude du taux d'expression des femmes selon le type des chaînes de télévisions

On s'intéresse maintenant au taux d'expression des femmes sur les chaînes de télévision selon le genre des chaînes. Pour cela, on récupère les données depuis un tableau de la page Wikipédia "Liste des chaînes de télévision en France" qui résument les informations nécessaires. 

In [ ]:
# enregistrement de l'url de la page d'intérêt.
url_channel="https://fr.wikipedia.org/wiki/Liste_des_cha%C3%AEnes_de_t%C3%A9l%C3%A9vision_en_France"
from urllib import request
request_text = request.urlopen(url_channel).read()

In [ ]:
# Utilisation de BeautifulSoup 

page = bs4.BeautifulSoup(request_text, "lxml")
# Récupération du tableau utile sur la page en code html.
table = page.find('table', {'class' : 'wikitable sortable'})
# On isole le corps du tableau.
channel_type=pd.read_html(table.prettify(), flavor="lxml")
df_channel_type=channel_type[0].drop(columns=["Logo"])
df_channel_type.head()

In [ ]:
data_channel_type = df_channel_type[['Nom','Genre']]
data_channel_type = data_channel_type.rename(columns = {'Nom':'channel_name','Genre':'channel_type'})
data_channel_type.head()

In [ ]:
df_tv = df_stats[df_stats["media_type"] == "tv"].copy()
df_tv.head(5)

In [ ]:
df_tv.loc[df_tv['channel_name'] == 'I-Télé/CNews', 'channel_name'] = 'CNews'
df_tv_type = df_tv.merge(data_channel_type,how='inner')
df_tv_type.sample(5)

In [ ]:
df_tv_type['channel_type'].unique()

In [ ]:
# On recrée le women_expression_rate
df_tv_type["women_expression_rate"] = df_tv_type[
    "female_duration"
] / (
    df_tv_type["female_duration"]
    + df_tv_type["male_duration"]
)
df_tv_type.sample(10)

In [ ]:
df_tv_channel_type = (
    df_tv_type.groupby(
        [
            "channel_type",
        ]
    )
    .agg(
        {
            "female_duration": "sum",
            "male_duration": "sum",
            "hour": "count",
        }
    )
    .reset_index()
    .rename(
    columns={
        "hour": "nb_hours_analyzed",
    }
    )
    )
df_tv_channel_type

In [ ]:
# On recrée le women_expression_rate
df_tv_channel_type["women_expression_rate"] = df_tv_channel_type[
    "female_duration"
] / (
    df_tv_channel_type["female_duration"]
    + df_tv_channel_type["male_duration"]
)
df_tv_channel_type

Le taux d'expression des femmes est très faibles sur les chaînes de sports (5%) ainsi que sur la châine 'Planète +' (17%), qui est la seule chaîne de l'échantillon classée dans la catégorie 'Découverte. Sur les chaînes classées dans les catégories 'Généraliste' et 'Information', les femmes s'expriment autant que dans la moyenne générale de l'échantillon (33-34%).

In [ ]:
df_tv_by_channel_type = ratio(
    df_tv_type.groupby(["channel_type", "channel_name"])
    .agg(
        {
            "female_duration": "sum",
            "male_duration": "sum",
            "music_duration": "sum",
            "hour": "sum",
        }
    )
    .reset_index()
    .rename(
    columns={
        "hour": "nb_hours_analyzed"
    })
).set_index(["channel_type", "channel_name"])

df_tv_by_channel_type

In [ ]:
channel_type = (
    ratio(
        df_tv_by_channel_type.groupby(["channel_type"])
        .agg(
            {
                "female_duration": "sum",
                "male_duration": "sum",
                "music_duration": "sum",
                "nb_hours_analyzed": "count",
            }
        )
        .reset_index()
    )
    .set_index(["channel_type"])[["women_expression_rate", "men_expression_rate"]]
)
channel_type

In [ ]:
channel_type.plot.bar(stacked=True)
plt.title("Taux d'expression masculin et féminin suivant le genre des chaînes tv")
plt.show()

In [ ]:
# création d'un nouveau dataframe contenant seulement les chaînes appartenant à France Télévions
df_tv_type['France Télévisions'] = df_tv_type['channel_name'].str.contains('France')
df_tv_type
df_france_tv = df_tv_type[df_tv_type['France Télévisions'] == True]
df_france_tv = df_france_tv[df_france_tv['year'] >= 2016]

In [ ]:
sns.barplot(x="channel_name", y="women_expression_rate",hue='year', data=df_france_tv)
plt.show()

Le taux d'expression des femmes a très légèrement augmenté entre 2016 et 2019 pour les chaînes appartenant au groupe France Télévisions. Cependant, ce taux reste très loin de la parité hommes-femmes. 

### Etude de la relation entre le taux d'expression des femmes et le taux d'audience des chaînes télévisées

In [ ]:
audiences = pd.read_excel(
    "data/audiences_tv.xlsx", 
    sheet_name = "PartdAudience", 
    skiprows=6, 
    nrows=35
)

audiences = audiences.rename(columns={audiences.columns[0]: "year"})

flattened_audiences = audiences.melt(id_vars=["year"], var_name="channel_name", value_name="audience")

flattened_audiences["audience"] = pd.to_numeric(flattened_audiences["audience"], errors="coerce")

df_audience = flattened_audiences.dropna(subset=["audience"])
df_audience.head(5)

In [ ]:
# Ensuite, on merge cette nouvelle base avec notre base de données sur les chaînes tv
grouped = df_year_level.groupby(df_year_level.media_type)
df_tv = grouped.get_group("tv").reset_index(drop = True)
df_tv_audience=df_tv.merge(df_audience,how='inner')
df_tv_audience['channel_name'].unique() #15 chaînes restantes
df_tv_audience.sample(5)

In [ ]:
plt.scatter(df_tv_audience['women_expression_rate'],df_tv_audience['audience'])
plt.show()

Il ne semble pas y avoir de corrélation entre le taux d'audience des chaînes et le taux d'expression des femmes sur ces chaînes.

In [ ]:
# statistiques decriptives
df_tv_audience['audience'].describe()

In [ ]:
#Grâce aux statistiques descriptives, on définit un seuil à partir duquel la chaîne est une chaîne à forte audience
# on choisit le 3ème quartile
# on fait de même pour définir des chaînes à faible audience et choix de la médiane comme seuil
df_tv_audience['high_audience']=df_tv_audience['audience']>9.4
df_tv_audience['low_audience']=df_tv_audience['audience']<2.8
df_tv_audience.head()

In [ ]:
# Exercice supplémentaire: classement des chaines par audience
df_tv_audience.loc[df_tv_audience['audience']>9.4,['channel_name']]
#On obtient : France 2, France 3, M6 et TF1

# Modélisation 

On dépasse désormais le descriptif et l'analyse. On choisit de prédire l'évolution mensuelle du _women expression rate_ pour une chaine donnée et pour un nombre de mois après la date de la dernière information certaine. Pour cela on utilise le package **Prophet**.

Prophet est une une bibliothèque (Python et R) de prévision de séries chronologiques basée sur un modèle additif où les tendances non linéaires sont ajustées à la saisonnalité annuelle notamment. 

L'input est une base de donnée à deux colonnes : `ds` et `y`. `y` (target) est la colonne numérique sur laquelle on veut obtenir le forecast. `ds` (datastamp column) est la colonne qui représente une date ou un instant.

In [ ]:
# TOP 5 en nombre d'heures analysées pour sélectionner les chaines sur lesquelles appliquer le forecast
df_stats.groupby(["channel_name"]).agg({"hour": "count"}).reset_index().sort_values(
    by="hour", ascending=False
).head(5)

## Préparation des données pour construire un modèle

In [ ]:
df_forecast = df_stats.copy()
df_forecast["date"] = pd.to_datetime(df_forecast["date"])

# On arrondit les dates au premier jour du mois
df_forecast["date"] = df_forecast["date"].dt.to_period('M').dt.to_timestamp()
df_forecast_grouped = (
    df_forecast.groupby(
        [
            "media_type",
            "channel_name",
            "is_public_channel",
            "date",
        ]
    )
    .agg(
        {
            "male_duration": "sum",
            "female_duration": "sum",
            "music_duration": "sum",
        }
    )
    .reset_index()
)

# On applique la fonction ratio 
df_forecast_grouped = ratio(df_forecast_grouped)

# Simplification de la base de données : on ne garde que les trois colonnes date, channel_name et women_expression_rate
df_forecast_grouped = df_forecast_grouped[
    ["date", "channel_name", "women_expression_rate"]
].copy()

df_forecast_grouped

## Modèle : _time series_ forecasting avec Prophet

In [ ]:
def forecast(channel_name, periods=12):
    """
    Prend en input le nom d'une chaîne
    et le nombre de périodes (mois) sur lesquelles 
    on veut prédire et affiche la prévision
    """
    # On filtre la base de données sur la chaîne passée en argument de la fonction.
    df_fit = df_forecast_grouped[
        df_forecast_grouped["channel_name"] == channel_name
    ].copy() 
    df_fit = df_fit[["date", "women_expression_rate"]].copy()
    # On renomme les colonnes de df_fit conformément aux attentes du modèle.
    df_fit.columns = [
        "ds",
        "y",
    ]

    try:
        # On initialise Prophet.
        m = Prophet(weekly_seasonality=False, daily_seasonality=False)
        
        # On ajoute les jours fériés français. 
        # Si le modèle pense qu’il est pertinent de les prendre en compte dans l'analyse, il le fera. 
        m.add_country_holidays(country_name="FR")
        
        # On applique le modèle à df_fit.
        m.fit(df_fit)
    
    except ValueError:
        print("Erreur : Chaîne non trouvée")
        return

    # On crée un dataframe future contenant les dates sur lesquelles les prévisions seront faites.
    future = m.make_future_dataframe(periods=periods, freq="MS")

    # On réalise la prédiction
    forecast = m.predict(future)

    fig1 = m.plot(forecast)
    fig2 = m.plot_components(forecast)

In [ ]:
forecast("France Culture", periods=24)

France Culture est choisie car la chaine fait partie du TOP 5 des chaînes les plus analysées. 

Forecast : les points noirs correspondent aux valeurs réelles du women_expression_rate pour la chaine France Culture. La courbe bleue foncée correpond au forecast, tous les composants (trend, saisonalité, résidus) de la série temporelle confondus. La qualité du forecast a l'air plutôt bonne: seuls quelques points sont en dehors de la zone bleues claires qui représente l'intervelle de confiance. 

Tendance : La tendance est décroissante de 1995 à 2005 avant de devenir fortement croissante à partir de 2005.

Les vacances scolaires : Prophet prend en compte également les jours fériés. On constate que les effets des vacances scolaires sont minimes. 

Saisonalité : On a introduit une saisonalité annuelle, sinon le forecast semble moins bon. Plus de valeurs réelle de y sont à l'extérieur de l'intervalle bleu.

In [ ]:
forecast("BFM TV", periods=36)

## Évaluation de la qualité du modèle : évaluation de la performance des prédictions

Il s'agit d'une "cross validation procedure". Prophet comprend une fonctionnalité d'évaluation de la qualité du modèle en mesurant l'erreur de prévision à l'aide de données historiques. Cela se fait en sélectionnant des points de coupure dans l'histoire, et pour chacun d'eux, en ajustant le modèle en utilisant des données uniquement jusqu'à ce point de coupure. On peut alors comparer les valeurs prévues aux valeurs réelles.

La sortie de la validation croisée est un cadre de données avec les valeurs réelles y et les valeurs de prévision hors échantillon $\hat{y}$, à chaque date de prévision simulée et pour chaque date limite. En particulier, une prévision est faite pour chaque point observé entre la date limite et la date limite + l'horizon. La nouvelle dataframe obtenue s'appelle df_cv. Ce cadre de données peut ensuite être utilisé pour calculer les mesures d'erreur de $\hat{y}$ par rapport à y. Ces mesures d'erreurs sont résumées dans d_p. 

Dans les forecast précédents, le modèle utilise pour une chaine donnée, toutes les données disponibles dans df_forecast (copie df_stats) pour prédire le women_expression_rate sur deux nouvelles années qui ne sont pas disponibles dans les données : 2019 et 2020. Toutes les chaines n'ayant pas le même nombres de mois disponibles dans les données, pour réaliser l'évaluation du modèle, on prend un cas particulier : on choisit comme paramètres, dans cross validation, initial = '6 Y', period='1 Y', horizon = '2 Y'. Signification : on a regardé si le modèle est capable de faire des prédictions performantes sur deux ans en prenant les 6 années précédant le cutoff comme historique. Ce test est fait autant de fois possibles sur les données en déplacant le cutoff de un an à chaque fois. 

In [ ]:
# Diagnostics 

def cross_validate(channel_name):
    """
    Prend en input le nom d'une chaîne
    et affiche deux dataframe comportant 
    des mesures de performance des prédictions
    """
    # On filtre la base de données sur la chaîne passée en argument de la fonction.
    df_fit = df_forecast_grouped[
        df_forecast_grouped["channel_name"] == channel_name
    ].copy() 
    df_fit = df_fit[["date", "women_expression_rate"]].copy()
    # On renomme les colonnes de df_fit conformément aux attentes du modèle.
    df_fit.columns = [
        "ds",
        "y",
    ]

    try:
        # On initialise Prophet.
        m = Prophet(weekly_seasonality=False, daily_seasonality=False)
        
        # On ajoute les jours fériés français. 
        # Si le modèle pense qu’il est pertinent de les prendre en compte dans l'analyse, il le fera. 
        m.add_country_holidays(country_name="FR")
        
        # On applique le modèle à df_fit.
        m.fit(df_fit)
    
    except ValueError:
        print("Erreur : Chaîne non trouvée")
        return
    
    df_cv = cross_validation(m, initial='312 W', period='52 W', horizon = '104 W')
    df_p = performance_metrics(df_cv)
    return df_cv, df_p

In [ ]:
df_cv, df_p = cross_validate("France Culture")

In [ ]:
df_cv

In [ ]:
df_p

In [ ]:
fig_mape = plot_cross_validation_metric(df_cv, metric='mape', rolling_window=0.1)

Les points montrent le % absolu d'erreurs pour chaque prédiction de df_cv. La ligne bleue indique le Mean Absolute Percentage error, où la moyenne est prise sur une fenêtre roulante des points. Pour cette prévision, on constate des erreurs d'environ 8 % qui sont typiques pour les prévisions sur ce type d'horizon.